In [2]:
def function_1(input_1):
    return input_1 + " Hi "

def function_2(input_2):
    return input_2 + "there"

In [3]:
from langgraph.graph import Graph

# Define a Langchain graph
workflow = Graph()

workflow.add_node("node_1", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge('node_1', 'node_2')

workflow.set_entry_point("node_1")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [4]:
app.invoke("Hello")

'Hello Hi there'

In [5]:
input = 'Hello'
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'node_1':
---
Hello Hi 

---

Output from node 'node_2':
---
Hello Hi there

---



In [6]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Now you can access your environment variables using os.environ
os.environ['OPENAI_API_KEY'] = os.environ.get("OPENAI_API_KEY")

In [7]:
from langchain_openai import ChatOpenAI

# Set the model as ChatOpenAI
model = ChatOpenAI(temperature=0) 

#Call the model with a user message
model.invoke('Hey there')

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-622d7af5-2730-475b-b479-af7c1335eb12-0', usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18})

In [8]:
model.invoke('Hey there').content

'Hello! How can I assist you today?'

In [9]:
def function_1(input_1):
    response = model.invoke(input_1)
    return response.content

def function_2(input_2):
    return "Agent Says: " + input_2

In [10]:
# Define a Langchain graph
workflow = Graph()

#calling node 1 as agent
workflow.add_node("agent", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge('agent', 'node_2')

workflow.set_entry_point("agent")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [11]:
app.invoke("Hey there")

'Agent Says: Hello! How can I assist you today?'

In [12]:
def function_1(input_1):
    complete_query = "Your task is to provide only the city name based on the user query. \
        Nothing more, just the city name mentioned. Following is the user query: " + input_1
    response = model.invoke(complete_query)
    return response.content

def function_2(input_2):
    return "Agent Says: " + input_

In [13]:
# Define a Langchain graph
workflow = Graph()

#calling node 1 as agent
workflow.add_node("agent", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge('agent', 'node_2')

workflow.set_entry_point("agent")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [14]:
def function_1(input_1):
    complete_query = "Your task is to provide only the city name based on the user query. \
        Nothing more, just the city name mentioned. Following is the user query: " + input_1
    response = model.invoke(complete_query)
    return response.content

def function_2(input_2):
    return "Agent Says: " + input_2

In [15]:
# Define a Langchain graph
workflow = Graph()

#calling node 1 as agent
workflow.add_node("agent", function_1)
workflow.add_node("node_2", function_2)

workflow.add_edge('agent', 'node_2')

workflow.set_entry_point("agent")
workflow.set_finish_point("node_2")

app = workflow.compile()

In [16]:
app.invoke("What's the temperature in Houston")

'Agent Says: Houston'

In [17]:
from langchain_community.utilities import OpenWeatherMapAPIWrapper
load_dotenv()
os.environ["OPENWEATHERMAP_API_KEY"] = os.environ.get("OPENWEATHERMAP_API_KEY")

weather = OpenWeatherMapAPIWrapper()

In [18]:
weather_data = weather.run("Houston")
print(weather_data)

In Houston, the current weather is as follows:
Detailed status: clear sky
Wind speed: 2.06 m/s, direction: 0°
Humidity: 50%
Temperature: 
  - Current: 36.14°C
  - High: 37.38°C
  - Low: 34.5°C
  - Feels like: 43.14°C
Rain: {}
Heat index: None
Cloud cover: 0%


In [19]:
def function_1(input_1):
    complete_query = "Your task is to provide only the city name based on the user query. \
        Nothing more, just the city name mentioned. Following is the user query: " + input_1
    response = model.invoke(complete_query)
    return response.content

def function_2(input_2):
    weather_data = weather.run(input_2)
    return weather_data

In [20]:
from langgraph.graph import Graph

workflow = Graph()

#calling node 1 as agent
workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

workflow.add_edge('agent', 'tool')

workflow.set_entry_point("agent")
workflow.set_finish_point("tool")

app = workflow.compile()

In [21]:
app.invoke("What's the temperature in Houston")

'In Houston, the current weather is as follows:\nDetailed status: clear sky\nWind speed: 2.06 m/s, direction: 0°\nHumidity: 50%\nTemperature: \n  - Current: 36.14°C\n  - High: 37.38°C\n  - Low: 34.5°C\n  - Feels like: 43.14°C\nRain: {}\nHeat index: None\nCloud cover: 0%'

In [22]:
def function_3(input_3):
    complete_query = "Your task is to provide info concisely based on the user query. Following is the user query: " + "user input"
    response = model.invoke(complete_query)
    return response.content

In [23]:
# assign AgentState as an empty dict
AgentState = {}

# messages key will be assigned as an empty array. We will append new messages as we pass along nodes. 
AgentState["messages"] = []

In [24]:
AgentState

{'messages': []}

In [25]:
def function_1(state):
    messages = state['messages']
    user_input = messages[-1]
    complete_query = "Your task is to provide only the city name based on the user query. \
                    Nothing more, just the city name mentioned. Following is the user query: " + user_input
    response = model.invoke(complete_query)
    state['messages'].append(response.content) # appending AIMessage response to the AgentState
    return state

def function_2(state):
    messages = state['messages']
    agent_response = messages[-1]
    weather = OpenWeatherMapAPIWrapper()
    weather_data = weather.run(agent_response)
    state['messages'].append(weather_data)
    return state

def function_3(state):
    messages = state['messages']
    user_input = messages[0]
    available_info = messages[-1]
    agent2_query = "Your task is to provide info concisely based on the user query and the available information from the internet. \
                        Following is the user query: " + user_input + " Available information: " + available_info
    response = model.invoke(agent2_query)
    return response.content

In [26]:
from langgraph.graph import Graph

workflow = Graph()


workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)
workflow.add_node("responder", function_3)

workflow.add_edge('agent', 'tool')
workflow.add_edge('tool', 'responder')

workflow.set_entry_point("agent")
workflow.set_finish_point("responder")

app = workflow.compile()

In [27]:
inputs = {"messages": ["what is the temperature in houston"]}
app.invoke(inputs)

'The current temperature in Houston is 36.14°C with clear skies, a low of 34.5°C, and a high of 37.38°C. It feels like 43.14°C.'

In [28]:
input = {"messages": ["what is the temperature in houston"]}
for output in app.stream(input):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': ['what is the temperature in houston', 'Houston']}

---

Output from node 'tool':
---
{'messages': ['what is the temperature in houston', 'Houston', 'In Houston, the current weather is as follows:\nDetailed status: clear sky\nWind speed: 2.06 m/s, direction: 0°\nHumidity: 50%\nTemperature: \n  - Current: 36.14°C\n  - High: 37.38°C\n  - Low: 34.5°C\n  - Feels like: 43.14°C\nRain: {}\nHeat index: None\nCloud cover: 0%']}

---

Output from node 'responder':
---
The current temperature in Houston is 36.14°C with clear skies and a low of 34.5°C and a high of 37.38°C. The humidity is at 50% and there is no rain expected. The heat index is not applicable.

---



In [29]:
inputs = {"messages": ["how are you?"]}
app.invoke(inputs)

"I am an AI assistant, so I don't have feelings, but thank you for asking. In Istanbul, the current weather is clear with a temperature of 24.28°C, a wind speed of 7.2 m/s, and 68% humidity."

In [30]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [31]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools.openweathermap import OpenWeatherMapQueryRun
from langchain_core.utils.function_calling import convert_to_openai_function

tools = [OpenWeatherMapQueryRun()]

model = ChatOpenAI(temperature=0, streaming=True)
functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

In [32]:
def function_1(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

In [33]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tools)

def function_2(state):
    messages = state['messages']
    last_message = messages[-1] # this has the query we need to send to the tool provided by the agent

    parsed_tool_input = json.loads(last_message.additional_kwargs["function_call"]["arguments"])

    # We construct an ToolInvocation from the function_call and pass in the tool name and the expected str input for OpenWeatherMap tool
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=parsed_tool_input['__arg1'],
    )
    
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)

    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)

    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

In [34]:
def where_to_go(state):
    messages = state['messages']
    last_message = messages[-1]
    
    if "function_call" in last_message.additional_kwargs:
        return "continue"
    else:
        return "end"

In [75]:
# from langgraph.graph import Graph, END

# workflow = Graph()
from langgraph.graph import StateGraph, END
workflow = StateGraph(AgentState)

workflow.add_node("agent", function_1)
workflow.add_node("tool", function_2)

# The conditional edge requires the following info below.
# First, we define the start node. We use `agent`.
# This means these are the edges taken after the `agent` node is called.
# Next, we pass in the function that will determine which node is called next, in our case where_to_go().

workflow.add_conditional_edges("agent", where_to_go,{   # Based on the return from where_to_go
                                                        # If return is "continue" then we call the tool node.
                                                        "continue": "tool",
                                                        # Otherwise we finish. END is a special node marking that the graph should finish.
                                                        "end": END
                                                    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that if `tool` is called, then it has to call the 'agent' next. 
workflow.add_edge('tool', 'agent')

# Basically, agent node has the option to call a tool node based on a condition, 
# whereas tool node must call the agent in all cases based on this setup.

workflow.set_entry_point("agent")


app = workflow.compile()

In [36]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the temperature in las vegas")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the temperature in las vegas'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"Las Vegas"}', 'name': 'open_weather_map'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-6ca70df5-a549-46b7-8dea-d1af35d7b49a-0'),
  FunctionMessage(content='In Las Vegas, the current weather is as follows:\nDetailed status: clear sky\nWind speed: 6.17 m/s, direction: 190°\nHumidity: 13%\nTemperature: \n  - Current: 37.83°C\n  - High: 38.94°C\n  - Low: 36.88°C\n  - Feels like: 35.25°C\nRain: {}\nHeat index: None\nCloud cover: 0%', name='open_weather_map'),
  AIMessage(content='The current temperature in Las Vegas is 37.83°C. It is a clear sky with a low humidity of 13%. The temperature feels like 35.25°C.', response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-c3de2e95-8c94-4d23-b9a8-9b96a145def4-0')]}

In [37]:
inputs = {"messages": [HumanMessage(content="what is the temperature in las vegas")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"__arg1":"Las Vegas"}', 'name': 'open_weather_map'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-f634448d-c5c0-43a2-b50a-77ce4fcab7ed-0')]}

---

Output from node 'tool':
---
{'messages': [FunctionMessage(content='In Las Vegas, the current weather is as follows:\nDetailed status: clear sky\nWind speed: 6.17 m/s, direction: 190°\nHumidity: 13%\nTemperature: \n  - Current: 37.83°C\n  - High: 38.94°C\n  - Low: 36.88°C\n  - Feels like: 35.25°C\nRain: {}\nHeat index: None\nCloud cover: 0%', name='open_weather_map')]}

---

Output from node 'agent':
---
{'messages': [AIMessage(content='The current temperature in Las Vegas is 37.83°C. It is a clear sky with a low humidity of 13%. The temperature feels like 35.25°C.', response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-902676f4-d952-410a